In [46]:
import pandas as pd
import numpy as np
import import_ipynb
from unidecode import unidecode
import glob
import csv
import warnings
import datetime as dt

import validItems

warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
pd.set_option('display.max_columns', 500)


In [47]:
obras = glob.glob(r"*.xlsx")
obras

['L01_AB - PLAN-PROG-SEMANAL-OBRAS-30-01-23_05-02-23.xlsx',
 'L03_PLAN-PROG-SEMANAL-OBRAS- 30-01-2023 a 05-02-2023.xlsx',
 'L06_PLAN-PROG-SEMANAL-OBRAS-30-01-23-05-02-23_ARTESP_Rev00.xlsx']

In [48]:
def treat_list(valid_counties):
    municipalities_treated = []
    for i in range(len(valid_counties)):
        valid_counties[i] = unidecode(valid_counties[i])
        valid_counties[i] = valid_counties[i].lower()
        municipalities_treated.append(valid_counties[i])
    return municipalities_treated


def check_highway(valid_highways, worksheet_highways):
    for i in range(len(worksheet_highways)):
        if not worksheet_highways[i] in valid_highways:
            wrong_list.append(f"Na planilha: {obra[0:3]} a rodovia esta errada! | a rodovia: {worksheet_highways[i]} nao existe!")
            
        
def check_codes(valid_codes, worksheet_codes):
    for i in range(len(worksheet_codes)):
        if not worksheet_codes[i] in valid_codes:
            wrong_list.append(f"Na planilha: {obra[0:3]} o cod de servico esta errado! | o codigo de servico: {worksheet_codes[i]} nao existe!")


def check_counties(valid_counties, worksheet_counties):
    for i in range(len(worksheet_counties)):
        if not unidecode(worksheet_counties[i].strip()) in valid_counties:
            wrong_list.append(f"Na planilha: {obra[0:3]} o municipio esta errado! | o municipio: {worksheet_counties[i].strip()} nao e do estado de Sao Paulo!")
            

def check_coordinates(worksheet, collumn_name: str):
    comma = ","
    coordinates_list = []
    for i in range(len(worksheet)):
        coordinates_list.append(worksheet[collumn_name][i])
    comma_list = [ a for a in coordinates_list if comma in str(a)]

    if len(comma_list) != 0:
        for i in range(len(comma_list)):
            wrong_list.append(f"Na planilha: {obra[0:3]} a coordenada esta errada! | esta com , no valor: {comma_list[i]}")

def check_senses(worksheet, valid_senses):  
    for i in range(len(worksheet)):
        if not worksheet[i].strip() in valid_senses:
            wrong_list.append(f"Na planilha: {obra[0:3]} o sentido esta errado! | o sentido: {worksheet[i]} não existe!")


def check_batch(worksheet_list, valid_batch):
    for i in range(len(worksheet_list)):
        if not worksheet_list[i] in valid_batch:
            wrong_list.append(f"Na planilha: {obra[0:3]} o lote esta errado! | o lote: {worksheet_list[i]} nao existe!")


def check_empty(worksheet, collumn_name):
    empty_list = []
    value_empty = []
    for i in range(len(worksheet)):
        if worksheet[collumn_name][i]:
            empty_list.append(worksheet[collumn_name][i])

    for i in range(len(empty_list)):
        if "nan" in str(empty_list[i]):
            value_empty.append(str(empty_list[i]).strip())
    if len(value_empty) > 0:
        wrong_list.append(f'Na planilha: {obra[0:3]}, na coluna {collumn_name}, existem {len(value_empty)} campos vazios!')
            
def check_km(worksheet, collumn_name: str):
    dot = "."
    comma = ","
    km_list = []
    for i in range(len(worksheet)):
        km_list.append(worksheet[collumn_name][i])
    dot_list = [a for a in km_list if dot in str(a)]
    comma_list = [ a for a in km_list if comma in str(a)]

    if len(comma_list) != 0 or len(dot_list) != 0:
        if len(comma_list) > 0:
            for i in range(len(comma_list)):
                wrong_list.append(f"Na planilha: {obra[0:3]} | Erro de KM, O KM {comma_list[i]} esta com ,")
        else:
            for i in range(len(dot_list)):
                wrong_list.append(f"Na planilha: {obra[0:3]} | Erro de KM, O KM {dot_list[i]} esta com .")
        return True
    else:
        return False

In [49]:
cabecalho = ['LOTE', 'RODOVIA', 'CÓD. DO SERVIÇO', 'DETALHAMENTO DO SERVIÇO',
       'KM INICIAL', 'LAT KM INICIAL', 'LON KM INICIAL', 'KM FINAL',
       'LAT KM FINAL', 'LON KM FINAL', 'SENTIDO', 'DATA INÍCIO', 'HORA INÍCIO',
       'DATA FIM', 'HORA FIM', 'FI1', 'FI2', 'FI3', 'FI4', 'FI5', 'FI6',
       'FIAC', 'ALT.', 'EMPRESA RESPONSÁVEL',
       'RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA',
       'ART DO RESPONSÁVEL TÉCNICO', 'NÍVEL DE SERVIÇO C/ INTERVENÇÃO',
       'APOIO PMRV', 'MUNICÍPIO', 'OBSERVAÇÕES GERAIS']

In [54]:
df_lista = []


wrong_list = []
for obra in obras:
    print("")
    print("planilha: ", obra)
    extensao = obra.split('.')[-1]
    print("extensao: ", extensao)
    df = pd.read_excel(obra, sheet_name="DADOS", 
        dtype={'DATA INÍCIO': object, 'DATA FIM': object, 'OBSERVAÇÕES GERAIS': object, 'DETALHAMENTO DO SERVIÇO': object, 'EMPRESA RESPONSÁVEL': object,
        'RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA': object, 'APOIO PMRV': object, 'MUNICÍPIO': object, 'ART DO RESPONSÁVEL TÉCNICO': object},
        names=cabecalho, usecols=[x for x in range(30)])
    cod_planilha = []
    rodovia_planilha = []
    municipios_planilha = []
    sentidos = []
    lote = []
    for i in range(len(df)):
        cod_planilha.append(df["CÓD. DO SERVIÇO"][i])
        rodovia_planilha.append(df["RODOVIA"][i])
        municipios_planilha.append(unidecode(df["MUNICÍPIO"][i].lower()))
        sentidos.append(df['SENTIDO'][i])
        lote.append(df["LOTE"][i])
    check_batch(lote, validItems.lotes)
    check_highway(validItems.rodovias, rodovia_planilha)
    check_codes(validItems.codigo_servico, cod_planilha)
    check_km(df, 'KM INICIAL')
    check_coordinates(df,"LAT KM INICIAL")
    check_coordinates(df,"LON KM INICIAL")
    check_km(df, 'KM FINAL')
    check_coordinates(df,"LAT KM FINAL")
    check_coordinates(df,"LON KM FINAL")
    check_senses(sentidos, validItems.sentido_validos)
    check_empty(df, "FI1")
    check_empty(df, "FI2")
    check_empty(df, "FI3")
    check_empty(df, "FI4")
    check_empty(df, "FI5")
    check_empty(df, "FI6")
    check_empty(df, "FIAC")
    check_empty(df, "ALT.")
    check_empty(df, "EMPRESA RESPONSÁVEL")
    check_empty(df, "RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA")
    check_empty(df, "ART DO RESPONSÁVEL TÉCNICO")
    check_empty(df, "NÍVEL DE SERVIÇO C/ INTERVENÇÃO")
    check_counties(treat_list(validItems.municipios), municipios_planilha)
    df_lista.append(df)

df['KM FINAL'] = df['KM FINAL'].apply(lambda valor: str(valor).replace('+', ''))
print(df_lista)
# if len(wrong_list) > 0:
#     print("Não podemos seguir por que a erros nas planilhas")
#     for i in range(len(wrong_list)):
#         print(wrong_list[i])
#         arquivo_erros = open("Erros.txt", 'a', encoding='utf8')
#         arquivo_erros.write(str(wrong_list[i]))
#         arquivo_erros.write("\n")

# decisao = input("Voce deseja continuar? (verificou os erros?) :")

# if decisao == "sim":
#     df = df[~(df.LOTE.isnull())]
#     data2 = pd.to_datetime(df['DATA FIM'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
#     df['DATA FIM'] = pd.to_datetime(df['DATA FIM'], errors='coerce', format='%d/%m/%Y')
#     df['DATA FIM'] = df['DATA FIM'].fillna(data2)    
#     data2 = pd.to_datetime(df['DATA INÍCIO'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
#     df['DATA INÍCIO'] = pd.to_datetime(df['DATA INÍCIO'], errors='coerce', format='%d/%m/%Y')
#     df['DATA INÍCIO'] = df['DATA INÍCIO'].fillna(data2)
#     df['KM INICIAL'] = df['KM INICIAL'].apply(lambda valor: str(valor).replace('+', ''))
#     df['KM FINAL'] = df['KM FINAL'].apply(lambda valor: str(valor).replace('+', ''))
#     df['KM INICIAL'] = df['KM INICIAL'].apply(lambda valor: str(valor).replace(',', ''))
#     df['KM FINAL'] = df['KM FINAL'].apply(lambda valor: str(valor).replace(',', ''))
#     df['KM INICIAL'] = pd.to_numeric(df['KM INICIAL'], errors='coerce')
#     df['KM FINAL'] = pd.to_numeric(df['KM FINAL'], errors='coerce')
#     df['KM INICIAL'] = df['KM INICIAL'] / 1000
#     df['KM FINAL'] = df['KM FINAL'] / 1000
#     df['OBSERVAÇÕES GERAIS'] = df['OBSERVAÇÕES GERAIS'].replace(r'\s+|\\n', ' ', regex=True)
#     df['DETALHAMENTO DO SERVIÇO'] = df['DETALHAMENTO DO SERVIÇO'].replace(r'\s+|\\n', ' ', regex=True)
#     df['EMPRESA RESPONSÁVEL'] = df['EMPRESA RESPONSÁVEL'].replace(r'\s+|\\n', ' ', regex=True)
#     df['RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA'] = df['RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA'].replace(r'\s+|\\n', ' ', regex=True)
#     df['APOIO PMRV'] = df['APOIO PMRV'].replace(r'\s+|\\n', ' ', regex=True)
#     df['MUNICÍPIO'] = df['MUNICÍPIO'].replace(r'\s+|\\n', ' ', regex=True)
#     df = df.iloc[:, :30]
#     df_rodovia_nulo = df['RODOVIA'].isnull()
#     df= df[~df_rodovia_nulo]
#     df_lista.append(df)
# else:
#     print("Corriga os erros depois tente novamente!")


planilha:  L01_AB - PLAN-PROG-SEMANAL-OBRAS-30-01-23_05-02-23.xlsx
extensao:  xlsx

planilha:  L03_PLAN-PROG-SEMANAL-OBRAS- 30-01-2023 a 05-02-2023.xlsx
extensao:  xlsx

planilha:  L06_PLAN-PROG-SEMANAL-OBRAS-30-01-23-05-02-23_ARTESP_Rev00.xlsx
extensao:  xlsx
[      LOTE RODOVIA  CÓD. DO SERVIÇO       DETALHAMENTO DO SERVIÇO KM INICIAL  \
0        2  SP 001                3  REPARO EM PAVIMENTO FLEXIVEL      62000   
1        1  SP 300                3  REPARO EM PAVIMENTO FLEXIVEL    062+100   
2        1  SP 300                3  REPARO EM PAVIMENTO FLEXIVEL    064+600   
3        1  SP 300                3  REPARO EM PAVIMENTO FLEXIVEL    064+600   
4        1  SP 300                3  REPARO EM PAVIMENTO FLEXIVEL    062+100   
...    ...     ...              ...                           ...        ...   
5994     1  SP 348              128                 PONTE/VIADUTO    048+000   
5995     1  SP 348              128                 PONTE/VIADUTO    048+000   
5996     1  SP 34

In [51]:
try:
       df_obras = pd.concat(df_lista, axis=0, ignore_index=True)
       df_obras.reset_index(drop=True, inplace=True)
       df_obras["LOTE"] = df_obras["LOTE"].astype(int)
       df_obras['MUNICÍPIO'] = df_obras['MUNICÍPIO'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
       df_obras['SENTIDO'] = df_obras['SENTIDO'].apply(lambda valor: str(valor).replace('OL','LO'))
       df_obras['SENTIDO'] = df_obras['SENTIDO'].apply(lambda valor: str(valor).replace('IE','EI'))
       df_obras['SENTIDO'] = df_obras['SENTIDO'].apply(lambda valor: str(valor).replace('SN','NS'))
       df_obras['SENTIDO'] = df_obras['SENTIDO'].apply(lambda valor: str(valor).replace('Leste','L'))
       df_obras['SENTIDO'] = df_obras['SENTIDO'].apply(lambda valor: str(valor).replace('Norte','N'))
       df_obras['SENTIDO'] = df_obras['SENTIDO'].apply(lambda valor: str(valor).replace('Oeste','O'))
       df_obras['SENTIDO'] = df_obras['SENTIDO'].apply(lambda valor: str(valor).replace('Sul','S'))
       df_obras['SENTIDO'] = df_obras['SENTIDO'].apply(lambda valor: str(valor).replace('Leste / Oeste','LO'))
       df_obras['SENTIDO'] = df_obras['SENTIDO'].apply(lambda valor: str(valor).replace('Norte / Sul','NS'))
       df_obras['SENTIDO'] = df_obras['SENTIDO'].apply(lambda valor: str(valor).replace('NN','N'))
       df_obras['SENTIDO'] = df_obras['SENTIDO'].apply(lambda valor: str(valor).replace('SS','S'))
       df_obras['MUNICÍPIO'] = df_obras['MUNICÍPIO'].apply(lambda valor: str(valor).replace('-SP',''))
       df_obras['MUNICÍPIO'] = df_obras['MUNICÍPIO'].apply(lambda valor: str(valor).replace(' -SP',''))
       df_obras['MUNICÍPIO'] = df_obras['MUNICÍPIO'].apply(lambda valor: str(valor).replace(' - SP',''))
       df_obras['MUNICÍPIO'] = df_obras['MUNICÍPIO'].apply(lambda valor: valor.split('/')[0])
       df_obras['NÍVEL DE SERVIÇO C/ INTERVENÇÃO'] = df_obras['NÍVEL DE SERVIÇO C/ INTERVENÇÃO'].replace(np.nan,'NÃO INFORMADO')
       df_obras['MUNICÍPIO'] = df_obras['MUNICÍPIO'].str.upper()
       df_obras['OBSERVAÇÕES GERAIS'] = df_obras['OBSERVAÇÕES GERAIS'].str.upper()
       df_obras['RODOVIA'] = df_obras['RODOVIA'].str.upper()
       df_obras['DETALHAMENTO DO SERVIÇO'] = df_obras['DETALHAMENTO DO SERVIÇO'].str.upper()
       df_obras['APOIO PMRV'] = df_obras['APOIO PMRV'].str.upper()
       df_obras['NÍVEL DE SERVIÇO C/ INTERVENÇÃO'] = df_obras['NÍVEL DE SERVIÇO C/ INTERVENÇÃO'].str.upper()
       df_obras['RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA'] = df_obras['RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA'].str.upper()
       df_obras['EMPRESA RESPONSÁVEL'] = df_obras['EMPRESA RESPONSÁVEL'].str.upper()
       df_obras['SENTIDO'] = df_obras['SENTIDO'].str.upper()
       df_obras['FI1'] = df_obras['FI1'].str.upper()
       df_obras['FI2'] = df_obras['FI2'].str.upper()
       df_obras['FI3'] = df_obras['FI3'].str.upper()
       df_obras['FI4'] = df_obras['FI4'].str.upper()
       df_obras['FI5'] = df_obras['FI5'].str.upper()
       df_obras['FI6'] = df_obras['FI6'].str.upper()
       df_obras['FIAC'] = df_obras['FIAC'].str.upper()
       df_obras['FI1'] = df_obras['FI1'].apply(lambda valor: str(valor).replace('S','1'))
       df_obras['FI2'] = df_obras['FI2'].apply(lambda valor: str(valor).replace('S','2'))
       df_obras['FI3'] = df_obras['FI3'].apply(lambda valor: str(valor).replace('S','3'))
       df_obras['FI4'] = df_obras['FI4'].apply(lambda valor: str(valor).replace('S','4'))
       df_obras['FI5'] = df_obras['FI5'].apply(lambda valor: str(valor).replace('S','5'))
       df_obras['FI6'] = df_obras['FI6'].apply(lambda valor: str(valor).replace('S','6'))
       df_obras['FIAC'] = df_obras['FIAC'].apply(lambda valor: str(valor).replace('S','ACOS'))
       df_obras['FAIXAS INTERDITADAS'] = df_obras['FI1'] + ',' + df_obras['FI2'] + ',' + df_obras['FI3'] + ','+ df_obras['FI4'] + ',' + df_obras['FI5'] + ',' + df_obras['FI6'] + ',' + df_obras['FIAC']
       df_obras['FAIXAS INTERDITADAS'] = df_obras['FAIXAS INTERDITADAS'].apply(lambda valor: str(valor).replace('N','NENHUMA'))
       df_obras['FAIXAS INTERDITADAS'] = df_obras['FAIXAS INTERDITADAS'].apply(lambda valor: str(valor).replace('6NENHUMA','6'))
       df_obras['FAIXAS INTERDITADAS'] = df_obras['FAIXAS INTERDITADAS'].apply(lambda valor: str(valor).replace('4NENHUMA','4'))
       df_obras['FAIXAS INTERDITADAS'] = df_obras['FAIXAS INTERDITADAS'].apply(lambda valor: str(valor).replace('3NENHUMA','3'))
       df_obras['FAIXAS INTERDITADAS'] = df_obras['FAIXAS INTERDITADAS'].apply(lambda valor: str(valor).replace('2NENHUMA','2'))
       df_obras['FAIXAS INTERDITADAS'] = df_obras['FAIXAS INTERDITADAS'].apply(lambda valor: str(valor).replace('1NENHUMA','1'))
       df_obras['FAIXAS INTERDITADAS'] = df_obras['FAIXAS INTERDITADAS'].apply(lambda valor: str(valor).replace('NENHUMA,',''))
       df_obras['FAIXAS INTERDITADAS'] = df_obras['FAIXAS INTERDITADAS'].apply(lambda valor: str(valor).replace('1,NENHUMA','1'))
       df_obras['FAIXAS INTERDITADAS'] = df_obras['FAIXAS INTERDITADAS'].apply(lambda valor: str(valor).replace('2,NENHUMA','2'))
       df_obras['FAIXAS INTERDITADAS'] = df_obras['FAIXAS INTERDITADAS'].apply(lambda valor: str(valor).replace('3,NENHUMA','3'))
       df_obras['FAIXAS INTERDITADAS'] = df_obras['FAIXAS INTERDITADAS'].apply(lambda valor: str(valor).replace('4,NENHUMA','4'))
       df_obras['FAIXAS INTERDITADAS'] = df_obras['FAIXAS INTERDITADAS'].apply(lambda valor: str(valor).replace('5,NENHUMA','5'))
       df_obras['FAIXAS INTERDITADAS'] = df_obras['FAIXAS INTERDITADAS'].apply(lambda valor: str(valor).replace('6,NENHUMA','6'))
       df_obras = df_obras[['LOTE', 'RODOVIA', 'CÓD. DO SERVIÇO', 'DETALHAMENTO DO SERVIÇO',
              'KM INICIAL', 'LAT KM INICIAL', 'LON KM INICIAL', 'KM FINAL',
              'LAT KM FINAL', 'LON KM FINAL', 'SENTIDO', 'DATA INÍCIO', 'HORA INÍCIO',
              'DATA FIM', 'HORA FIM', 'FI1', 'FI2', 'FI3', 'FI4', 'FI5', 'FI6',
              'FIAC', 'ALT.', 'FAIXAS INTERDITADAS' , 'EMPRESA RESPONSÁVEL',
              'RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA',
              'ART DO RESPONSÁVEL TÉCNICO', 'NÍVEL DE SERVIÇO C/ INTERVENÇÃO',
              'APOIO PMRV', 'MUNICÍPIO', 'OBSERVAÇÕES GERAIS']]
       df_obras.to_csv("OBRAS_26_12_2022_a_01_01_2023.csv", encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC, index=False)
except ValueError:
       print("Não passou pelas validações corriga para tentar novamente")